In [1]:
import pickle
import random
import numpy
import torch
import os

from Auxiliary_Functions.generate_observation import generate_observations, split_dataset
from Auxiliary_Functions.save_progress import save_progress
from Auxiliary_Functions.matrix_normalization import normalize_max_row_norm

from Denoising_Algorithms.Projectors.DnCNN import DnCNN
from Denoising_Algorithms.DL_Training.evaluation import average_PSNR
from Denoising_Algorithms.DL_Training.loss_functions import IntermediateScaledLoss
from Denoising_Algorithms.DL_Training.training import train_main
from Denoising_Algorithms.Memory_Network.Memory_Net import MemoryNetwork
from Denoising_Algorithms.DL_Training.evaluation import average_PSNR

In [2]:
current_trial = 0

torch.manual_seed(current_trial)
numpy.random.seed(current_trial)
random.seed(current_trial)
torch.cuda.manual_seed(current_trial)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/Gaussian_10.pkl', 'rb') as f:
    gaussian_10 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/Gaussian_20.pkl', 'rb') as f:
    gaussian_20 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/Gaussian_30.pkl', 'rb') as f:
    gaussian_30 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/Gaussian_40.pkl', 'rb') as f:
    gaussian_40 = pickle.load(f)

In [4]:
with open('ILSVRC2012_50x50_small.pkl', 'rb') as f:
    ILSVRC2012 = pickle.load(f)

# We only work with the first 25000 images
img_dataset = ILSVRC2012[:25000]

In [5]:
gaussian_10_normalized = normalize_max_row_norm(gaussian_10)
gaussian_20_normalized = normalize_max_row_norm(gaussian_20)
gaussian_30_normalized = normalize_max_row_norm(gaussian_30)
gaussian_40_normalized = normalize_max_row_norm(gaussian_40)

gaussian_10_observations = generate_observations(img_dataset, gaussian_10_normalized, 0.01)
gaussian_20_observations = generate_observations(img_dataset, gaussian_20_normalized, 0.01)
gaussian_30_observations = generate_observations(img_dataset, gaussian_30_normalized, 0.01)
gaussian_40_observations = generate_observations(img_dataset, gaussian_40_normalized, 0.01)

gaussian_10_train, gaussian_10_test = split_dataset(gaussian_10_observations, train_ratio = 0.9, seed = 0)
gaussian_20_train, gaussian_20_test = split_dataset(gaussian_20_observations, train_ratio = 0.9, seed = 0)
gaussian_30_train, gaussian_30_test = split_dataset(gaussian_30_observations, train_ratio = 0.9, seed = 0)
gaussian_40_train, gaussian_40_test = split_dataset(gaussian_40_observations, train_ratio = 0.9, seed = 0)

In [6]:
projector = DnCNN(depth = 10)

# Equal Weighting
loss_function = IntermediateScaledLoss(omega = 1.0)

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
# 5 Projections
num_projections = 5

# Folder Path for 5 Projections
proj5_path = 'Memory_Residual-Additive_Depth10-.01/5 Projections'

gauss_10_proj5_weights, gauss_10_proj5_hist = train_main(gaussian_10_normalized, [1] * num_projections, gaussian_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_10_proj5', display=True)

gauss_20_proj5_weights, gauss_20_proj5_hist = train_main(gaussian_20_normalized, [1] * num_projections, gaussian_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_20_proj5', display=True)

gauss_30_proj5_weights, gauss_30_proj5_hist = train_main(gaussian_30_normalized, [1] * num_projections, gaussian_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_30_proj5', display=True)

gauss_40_proj5_weights, gauss_40_proj5_hist = train_main(gaussian_40_normalized, [1] * num_projections, gaussian_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_40_proj5', display=True)

# Store Best Model Weights and History

save_progress(proj5_path, gauss_10_proj5_weights, gauss_10_proj5_hist, '10% Sampling')
save_progress(proj5_path, gauss_20_proj5_weights, gauss_20_proj5_hist, '20% Sampling')
save_progress(proj5_path, gauss_30_proj5_weights, gauss_30_proj5_hist, '30% Sampling')
save_progress(proj5_path, gauss_40_proj5_weights, gauss_40_proj5_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 0.28116231168640987
Average test loss: 0.013998244442045688
Epoch 2/300
Average training loss: 0.10253174093696805
Average test loss: 0.011639097789095507
Epoch 3/300
Average training loss: 0.08349368072880639
Average test loss: 0.014221046048733922
Epoch 4/300
Average training loss: 0.0753149357702997
Average test loss: 0.009103507315119108
Epoch 5/300
Average training loss: 0.0691623524957233
Average test loss: 0.015326453828977214
Epoch 6/300
Average training loss: 0.06374343168735504
Average test loss: 0.009467638175520631
Epoch 7/300
Average training loss: 0.06198756770292918
Average test loss: 0.008526010759174824
Epoch 8/300
Average training loss: 0.05813114857673645
Average test loss: 0.009317908874402444
Epoch 9/300
Average training loss: 0.056453960537910464
Average test loss: 0.009949641190469266
Epoch 10/300
Average training loss: 0.053942013674312166
Average test loss: 0.008711476021342807
Epoch 11

In [8]:
gauss_10_proj5_model = MemoryNetwork(gaussian_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_20_proj5_model = MemoryNetwork(gaussian_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_30_proj5_model = MemoryNetwork(gaussian_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_40_proj5_model = MemoryNetwork(gaussian_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

gauss_10_proj5_psnr = average_PSNR(gauss_10_proj5_model, gauss_10_proj5_weights, num_projections, gaussian_10_test, print_psnr=True)
gauss_20_proj5_psnr = average_PSNR(gauss_20_proj5_model, gauss_20_proj5_weights, num_projections, gaussian_20_test, print_psnr=True)
gauss_30_proj5_psnr = average_PSNR(gauss_30_proj5_model, gauss_30_proj5_weights, num_projections, gaussian_30_test, print_psnr=True)
gauss_40_proj5_psnr = average_PSNR(gauss_40_proj5_model, gauss_40_proj5_weights, num_projections, gaussian_40_test, print_psnr=True)

with open(os.path.join(proj5_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_10_proj5_psnr, f)
with open(os.path.join(proj5_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_20_proj5_psnr, f)
with open(os.path.join(proj5_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_30_proj5_psnr, f)
with open(os.path.join(proj5_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_40_proj5_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 22.39
Average PSNR for Projection Layer 1 across 2500 images: 24.18
Average PSNR for Projection Layer 2 across 2500 images: 25.02
Average PSNR for Projection Layer 3 across 2500 images: 25.65
Average PSNR for Projection Layer 4 across 2500 images: 26.07
Average PSNR for Projection Layer 0 across 2500 images: 23.13
Average PSNR for Projection Layer 1 across 2500 images: 25.84
Average PSNR for Projection Layer 2 across 2500 images: 27.34
Average PSNR for Projection Layer 3 across 2500 images: 28.24
Average PSNR for Projection Layer 4 across 2500 images: 28.79
Average PSNR for Projection Layer 0 across 2500 images: 24.25
Average PSNR for Projection Layer 1 across 2500 images: 27.25
Average PSNR for Projection Layer 2 across 2500 images: 29.04
Average PSNR for Projection Layer 3 across 2500 images: 29.84
Average PSNR for Projection Layer 4 across 2500 images: 30.51
Average PSNR for Projection Layer 0 across 2500 images: 24.44
Average 

In [9]:
# 15 Projections
num_projections = 15

# Folder Path for 15 Projections
proj15_path = 'Memory_Residual-Additive_Depth10-.01/15 Projections'

gauss_10_proj15_weights, gauss_10_proj15_hist = train_main(gaussian_10_normalized, [1] * num_projections, gaussian_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_10_proj15', display=True)

gauss_20_proj15_weights, gauss_20_proj15_hist = train_main(gaussian_20_normalized, [1] * num_projections, gaussian_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_20_proj15', display=True)

gauss_30_proj15_weights, gauss_30_proj15_hist = train_main(gaussian_30_normalized, [1] * num_projections, gaussian_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_30_proj15', display=True)

gauss_40_proj15_weights, gauss_40_proj15_hist = train_main(gaussian_40_normalized, [1] * num_projections, gaussian_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_40_proj15', display=True)

# Store Best Model Weights and History

save_progress(proj15_path, gauss_10_proj15_weights, gauss_10_proj15_hist, '10% Sampling')
save_progress(proj15_path, gauss_20_proj15_weights, gauss_20_proj15_hist, '20% Sampling')
save_progress(proj15_path, gauss_30_proj15_weights, gauss_30_proj15_hist, '30% Sampling')
save_progress(proj15_path, gauss_40_proj15_weights, gauss_40_proj15_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 5.159134792539809
Average test loss: 1.3152923698359065
Epoch 2/300
Average training loss: 2.477279816309611
Average test loss: 0.01593133513132731
Epoch 3/300
Average training loss: 1.8754397872289021
Average test loss: 0.012929846155146758
Epoch 4/300
Average training loss: 1.5318757762908937
Average test loss: 0.05299553495645523
Epoch 5/300
Average training loss: 1.266101944287618
Average test loss: 0.01793888136578931
Epoch 6/300
Average training loss: 1.0786390316221448
Average test loss: 0.031747606333759094
Epoch 7/300
Average training loss: 0.9375770379172431
Average test loss: 0.009311059937708907
Epoch 8/300
Average training loss: 0.8254615059428745
Average test loss: 0.010059488225314352
Epoch 9/300
Average training loss: 0.7320084744029575
Average test loss: 0.008659454209109147
Epoch 10/300
Average training loss: 0.6547607383198208
Average test loss: 0.007772641412085957
Epoch 11/300
Average train

In [10]:
gauss_10_proj15_model = MemoryNetwork(gaussian_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_20_proj15_model = MemoryNetwork(gaussian_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_30_proj15_model = MemoryNetwork(gaussian_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_40_proj15_model = MemoryNetwork(gaussian_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

gauss_10_proj15_psnr = average_PSNR(gauss_10_proj15_model, gauss_10_proj15_weights, num_projections, gaussian_10_test, print_psnr=True)
gauss_20_proj15_psnr = average_PSNR(gauss_20_proj15_model, gauss_20_proj15_weights, num_projections, gaussian_20_test, print_psnr=True)
gauss_30_proj15_psnr = average_PSNR(gauss_30_proj15_model, gauss_30_proj15_weights, num_projections, gaussian_30_test, print_psnr=True)
gauss_40_proj15_psnr = average_PSNR(gauss_40_proj15_model, gauss_40_proj15_weights, num_projections, gaussian_40_test, print_psnr=True)

with open(os.path.join(proj15_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_10_proj15_psnr, f)
with open(os.path.join(proj15_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_20_proj15_psnr, f)
with open(os.path.join(proj15_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_30_proj15_psnr, f)
with open(os.path.join(proj15_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_40_proj15_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 18.95
Average PSNR for Projection Layer 1 across 2500 images: 21.43
Average PSNR for Projection Layer 2 across 2500 images: 22.64
Average PSNR for Projection Layer 3 across 2500 images: 23.45
Average PSNR for Projection Layer 4 across 2500 images: 23.91
Average PSNR for Projection Layer 5 across 2500 images: 24.79
Average PSNR for Projection Layer 6 across 2500 images: 25.22
Average PSNR for Projection Layer 7 across 2500 images: 25.48
Average PSNR for Projection Layer 8 across 2500 images: 25.83
Average PSNR for Projection Layer 9 across 2500 images: 26.03
Average PSNR for Projection Layer 10 across 2500 images: 26.23
Average PSNR for Projection Layer 11 across 2500 images: 26.50
Average PSNR for Projection Layer 12 across 2500 images: 26.69
Average PSNR for Projection Layer 13 across 2500 images: 26.88
Average PSNR for Projection Layer 14 across 2500 images: 26.98
Average PSNR for Projection Layer 0 across 2500 images: 23.15
Ave

In [11]:
# 30 Projections
num_projections = 30

# Folder Path for 30 Projections
proj30_path = 'Memory_Residual-Additive_Depth10-.01/30 Projections'

gauss_10_proj30_weights, gauss_10_proj30_hist = train_main(gaussian_10_normalized, [1] * num_projections, gaussian_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_10_proj30', display=True)

gauss_20_proj30_weights, gauss_20_proj30_hist = train_main(gaussian_20_normalized, [1] * num_projections, gaussian_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_20_proj30', display=True)

gauss_30_proj30_weights, gauss_30_proj30_hist = train_main(gaussian_30_normalized, [1] * num_projections, gaussian_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_30_proj30', display=True)

gauss_40_proj30_weights, gauss_40_proj30_hist = train_main(gaussian_40_normalized, [1] * num_projections, gaussian_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_40_proj30', display=True)

# Store Best Model Weights and History

save_progress(proj30_path, gauss_10_proj30_weights, gauss_10_proj30_hist, '10% Sampling')
save_progress(proj30_path, gauss_20_proj30_weights, gauss_20_proj30_hist, '20% Sampling')
save_progress(proj30_path, gauss_30_proj30_weights, gauss_30_proj30_hist, '30% Sampling')
save_progress(proj30_path, gauss_40_proj30_weights, gauss_40_proj30_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 41.80465545993381
Average test loss: 0.027380966463022762
Epoch 2/300
Average training loss: 18.38431677585178
Average test loss: 0.03248022295203474
Epoch 3/300
Average training loss: 12.70553933207194
Average test loss: 14.102252251916461
Epoch 4/300
Average training loss: 8.915941240098741
Average test loss: 0.04537667570677068
Epoch 5/300
Average training loss: 7.112725441826714
Average test loss: 2.816195452047719
Epoch 6/300
Average training loss: 6.1548004578484425
Average test loss: 0.01214336854716142
Epoch 7/300
Average training loss: 4.956765889909533
Average test loss: 1.427789199716515
Epoch 8/300
Average training loss: 4.542490794711643
Average test loss: 138.58228542594114
Epoch 9/300
Average training loss: 4.10451442888048
Average test loss: 0.03119078525652488
Epoch 10/300
Average training loss: 3.9308577325608995
Average test loss: 200998.91211582502
Epoch 11/300
Average training loss: 3.39858

In [12]:
gauss_10_proj30_model = MemoryNetwork(gaussian_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_20_proj30_model = MemoryNetwork(gaussian_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_30_proj30_model = MemoryNetwork(gaussian_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_40_proj30_model = MemoryNetwork(gaussian_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

gauss_10_proj30_psnr = average_PSNR(gauss_10_proj30_model, gauss_10_proj30_weights, num_projections, gaussian_10_test, print_psnr=True)
gauss_20_proj30_psnr = average_PSNR(gauss_20_proj30_model, gauss_20_proj30_weights, num_projections, gaussian_20_test, print_psnr=True)
gauss_30_proj30_psnr = average_PSNR(gauss_30_proj30_model, gauss_30_proj30_weights, num_projections, gaussian_30_test, print_psnr=True)
gauss_40_proj30_psnr = average_PSNR(gauss_40_proj30_model, gauss_40_proj30_weights, num_projections, gaussian_40_test, print_psnr=True)

with open(os.path.join(proj30_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_10_proj30_psnr, f)
with open(os.path.join(proj30_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_20_proj30_psnr, f)
with open(os.path.join(proj30_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_30_proj30_psnr, f)
with open(os.path.join(proj30_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_40_proj30_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 20.08
Average PSNR for Projection Layer 1 across 2500 images: 21.86
Average PSNR for Projection Layer 2 across 2500 images: 22.60
Average PSNR for Projection Layer 3 across 2500 images: 23.43
Average PSNR for Projection Layer 4 across 2500 images: 24.11
Average PSNR for Projection Layer 5 across 2500 images: 24.39
Average PSNR for Projection Layer 6 across 2500 images: 24.77
Average PSNR for Projection Layer 7 across 2500 images: 25.11
Average PSNR for Projection Layer 8 across 2500 images: 25.38
Average PSNR for Projection Layer 9 across 2500 images: 25.37
Average PSNR for Projection Layer 10 across 2500 images: 25.40
Average PSNR for Projection Layer 11 across 2500 images: 25.55
Average PSNR for Projection Layer 12 across 2500 images: 25.87
Average PSNR for Projection Layer 13 across 2500 images: 25.96
Average PSNR for Projection Layer 14 across 2500 images: 25.89
Average PSNR for Projection Layer 15 across 2500 images: 26.15
Av